スクレイピング

In [49]:
%pip install lxml
%pip install beautifulsoup4
%pip install requests
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [50]:
import requests
from bs4 import BeautifulSoup
import re
from bs4 import XMLParsedAsHTMLWarning
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)


In [51]:
jcd = "01" # ボートレース場のコード
hd = "20251129" # 日付（YYYYMMDD形式）
rno = "12" # レース番号

In [52]:
raceid = hd + jcd + rno.zfill(2)

# 出走表

In [53]:
racelist_prefix = "https://www.boatrace.jp/owpc/pc/race/racelist?"

In [54]:
racelist_url = f"{racelist_prefix}rno={rno}&jcd={jcd}&hd={hd}"
racelist_url

'https://www.boatrace.jp/owpc/pc/race/racelist?rno=12&jcd=01&hd=20251129'

In [55]:
racelist_response = requests.get(racelist_url)
racelist_response.raise_for_status()
racelist_soup = BeautifulSoup(racelist_response.content, 'lxml')

In [91]:
race_header = racelist_soup.find('h3', class_='title16_titleDetail__add2020')
if race_header:
    # ヘッダー内のテキストを取得（余分な空白を除去）
    header_text = race_header.get_text(strip=True)
    # 正規表現で「数字 + m」のパターン（例: 1800m）を抽出
    match = re.search(r'(\d{3,4}m)', header_text)
    course_length = match.group(1)[:-1]

In [57]:
racelist_racer_containers = racelist_soup.find_all('tbody', class_='is-fs12')

In [ ]:
class Racer_racelist_info:
    def __init__(self, racer_container):
        racer_basic_infos = racer_container.get_text().split()
        self.wakuban, self.racer_id, _, self.grade, _, _, _, weight_age, self.f_num, self.l_num, self.st_ave, self.g_winrate, self.g_2rate, self.g_3rate, self.l_winrate, self.l_2rate, self.l_3rate, self.m_id, self.m_2rate, self.m_3rate, self.b_id, self.b_2rate, self.b_3rate = racer_basic_infos[:23]
        match = re.search(r'(\d+)歳/(\d+.\d)kg', weight_age) 
        self.weight = match.group(1) if match else None
        self.age = match.group(2) if match else None

    def print(self):
        print(*self.__dict__.values())

In [59]:
racer_racelist_info_list = []
for container in racelist_racer_containers:
    racer_racelist_info_list.append(Racer_racelist_info(container))

直前情報

In [ ]:
before_info_prefix = "https://www.boatrace.jp/owpc/pc/race/beforeinfo?"
before_info_url = f"{before_info_prefix}rno={rno}&jcd={jcd}&hd={hd}"
before_info_url

'https://www.boatrace.jp/owpc/pc/race/beforeinfo?rno=12&jcd=01&hd=20251129'

In [ ]:
before_info_response = requests.get(before_info_url)
before_info_response.raise_for_status()
before_info_soup = BeautifulSoup(before_info_response.content, 'lxml')

In [ ]:
before_info_racer_containers = before_info_soup.find_all('tbody', class_='is-fs12')

In [63]:
class Racer_before_info:
    def __init__(self, racer_container):
        racer_before_infos = racer_container.get_text().split()
        self.wakuban, _, _, _, self.disp_time, self.tilt, _, _, _, _, self.adj_weight= racer_before_infos[:11]

    def print(self):
        print(*self.__dict__.values())

In [ ]:
racer_before_info_list = []
for container in before_info_racer_containers:
    racer_before_info_list.append(Racer_before_info(container))

In [65]:
result_prefix = "https://www.boatrace.jp/owpc/pc/race/raceresult?"
result_url = f"{result_prefix}rno={rno}&jcd={jcd}&hd={hd}"

In [66]:
result_response = requests.get(result_url)
result_response.raise_for_status()
result_soup = BeautifulSoup(result_response.content, 'lxml')

In [67]:
result_table_container = result_soup.find('table', class_='is-w495')
result_rows = result_table_container.find_all('tbody')
result_row = result_rows[0].get_text().split()
result_row


['１', '1', '3532', '柴田', '光', '1\'47"1']

In [68]:
class Race_result_info:
    def __init__(self, result_row):
        result_infos = result_row.get_text().split()
        self.rank, self.wakuban, _, _, _, self.time = result_infos[:6]
    def print(self):
        print(*self.__dict__.values())

In [69]:
race_result_info_list = []
for result_row in result_rows:
    race_result_info_list.append(Race_result_info(result_row))

In [70]:
racer_racelist_info_list, racer_before_info_list, race_result_info_list

([<__main__.Racer_racelist_info at 0x2dc47727050>,
  <__main__.Race_result_info at 0x2dc47b18350>])

In [71]:
racelist_df = pd.DataFrame([r.__dict__ for r in racer_racelist_info_list])
before_df = pd.DataFrame([r.__dict__ for r in racer_before_info_list])
result_df = pd.DataFrame([r.__dict__ for r in race_result_info_list])

In [77]:
racelist_df, before_df, result_df

(  wakuban    id grade f_num l_num st_ave g_winrate g_2rate g_3rate l_winrate  \
 0       1  3532    A1    F0    L0   0.16      6.97   50.41   73.17      6.41   
 1       2  3576    A1    F0    L0   0.17      6.45   47.47   69.70      0.00   
 2       3  4743    A1    F0    L0   0.14      6.72   49.23   69.23      6.06   
 3       4  3898    A1    F0    L0   0.14      6.73   52.38   71.43      6.83   
 4       5  4357    A1    F0    L0   0.17      6.63   51.30   66.09      6.90   
 5       6  4798    A1    F0    L0   0.13      6.42   42.62   65.57      6.06   
 
   l_2rate l_3rate m_id m_2rate m_3rate b_id b_2rate b_3rate weight   age  
 0   48.03   66.45   74   36.36   56.28   73    0.00    0.00     53  52.0  
 1    0.00    0.00   20   39.42   55.29   67    0.00    0.00     53  52.0  
 2   43.75   56.25   24   38.29   56.00   34   14.29   57.14     33  53.4  
 3   50.00   66.67   38   29.08   46.94   71    0.00    0.00     49  54.4  
 4   50.00   80.00   52   49.00   67.00   36    0.0

In [82]:
racelist_df["wakuban"] = racelist_df["wakuban"].astype(str).str.normalize('NFKC')
result_df["rank"] = result_df["rank"].astype(str).str.normalize('NFKC')
racelist_df, result_df

(  wakuban    id grade f_num l_num st_ave g_winrate g_2rate g_3rate l_winrate  \
 0       1  3532    A1    F0    L0   0.16      6.97   50.41   73.17      6.41   
 1       2  3576    A1    F0    L0   0.17      6.45   47.47   69.70      0.00   
 2       3  4743    A1    F0    L0   0.14      6.72   49.23   69.23      6.06   
 3       4  3898    A1    F0    L0   0.14      6.73   52.38   71.43      6.83   
 4       5  4357    A1    F0    L0   0.17      6.63   51.30   66.09      6.90   
 5       6  4798    A1    F0    L0   0.13      6.42   42.62   65.57      6.06   
 
   l_2rate l_3rate m_id m_2rate m_3rate b_id b_2rate b_3rate weight   age  
 0   48.03   66.45   74   36.36   56.28   73    0.00    0.00     53  52.0  
 1    0.00    0.00   20   39.42   55.29   67    0.00    0.00     53  52.0  
 2   43.75   56.25   24   38.29   56.00   34   14.29   57.14     33  53.4  
 3   50.00   66.67   38   29.08   46.94   71    0.00    0.00     49  54.4  
 4   50.00   80.00   52   49.00   67.00   36    0.0

In [105]:
df = pd.merge(racelist_df, before_df, on='wakuban', how='left')
df = pd.merge(df, result_df, on='wakuban', how='left')

In [106]:
df["race_id"] = raceid
df["course_length"] = course_length

In [107]:
df

,wakuban,id,grade,f_num,l_num,st_ave,g_winrate,g_2rate,g_3rate,l_winrate,...,b_3rate,weight,age,disp_time,tilt,adj_weight,rank,time,race_id,course_length
0,1,3532,A1,F0,L0,0.16,6.97,50.41,73.17,6.41,...,0.00,53,52.0,6.92,-0.5,0.0,1,"1'47""1",202511290112,1800
1,2,3576,A1,F0,L0,0.17,6.45,47.47,69.70,0.00,...,0.00,53,52.0,6.82,-0.5,0.0,6,"1'55""4",202511290112,1800
2,3,4743,A1,F0,L0,0.14,6.72,49.23,69.23,6.06,...,57.14,33,53.4,6.71,0.0,0.0,2,"1'49""7",202511290112,1800
3,4,3898,A1,F0,L0,0.14,6.73,52.38,71.43,6.83,...,0.00,49,54.4,6.89,-0.5,0.0,4,"1'52""1",202511290112,1800
4,5,4357,A1,F0,L0,0.17,6.63,51.30,66.09,6.90,...,28.57,41,52.8,6.81,-0.5,0.0,3,"1'50""8",202511290112,1800
5,6,4798,A1,F0,L0,0.13,6.42,42.62,65.57,6.06,...,0.00,32,52.7,6.73,-0.5,0.0,5,"1'53""8",202511290112,1800


In [110]:
def convert_time_to_seconds(time_str):
    try:
        time_str = time_str.replace('\"', '\'')
        minutes, seconds, deciseconds = time_str.split('\'')
        total_seconds = int(minutes) * 60 + int(seconds) + int(deciseconds) / 10
        return total_seconds
    except ValueError:
        return np.nan

In [ ]:
df['time'] = df['time'].apply(convert_time_to_seconds)
df['f_num'] = df['f_num'].str.extract(r'(\d+)')
df['f_num'] = df['f_num'].astype(int)
df['l_num'] = df['l_num'].str.extract(r'(\d+)')
df['l_num'] = df['l_num'].astype(int)

In [120]:
df

,wakuban,id,grade,f_num,l_num,st_ave,g_winrate,g_2rate,g_3rate,l_winrate,...,weight,age,disp_time,tilt,adj_weight,rank,time,race_id,course_length,time_seconds
0,1,3532,A1,0,0,0.16,6.97,50.41,73.17,6.41,...,53,52.0,6.92,-0.5,0.0,1,"1'47""1",202511290112,1800,107.1
1,2,3576,A1,0,0,0.17,6.45,47.47,69.70,0.00,...,53,52.0,6.82,-0.5,0.0,6,"1'55""4",202511290112,1800,115.4
2,3,4743,A1,0,0,0.14,6.72,49.23,69.23,6.06,...,33,53.4,6.71,0.0,0.0,2,"1'49""7",202511290112,1800,109.7
3,4,3898,A1,0,0,0.14,6.73,52.38,71.43,6.83,...,49,54.4,6.89,-0.5,0.0,4,"1'52""1",202511290112,1800,112.1
4,5,4357,A1,0,0,0.17,6.63,51.30,66.09,6.90,...,41,52.8,6.81,-0.5,0.0,3,"1'50""8",202511290112,1800,110.8
5,6,4798,A1,0,0,0.13,6.42,42.62,65.57,6.06,...,32,52.7,6.73,-0.5,0.0,5,"1'53""8",202511290112,1800,113.8
